In [3]:
import pandas as pd
import hashlib

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

N_PARTITIONS = 1000

In [4]:
def id_to_hash(customer_id):
    return int(hashlib.md5(customer_id.encode('utf-8')).hexdigest(), 16)

In [5]:
members = pd.read_csv('/data/churn/members_v3.csv', nrows = 1)
members

transactions = pd.read_csv('/data/churn/all_trans.csv', nrows = 1)
transactions

logs = pd.read_csv('/data/churn/user_logs.csv', nrows = 1)
logs

train = pd.read_csv('/data/churn/all_train.csv', nrows = 1)
train

test = pd.read_csv('/data/churn/sample_submission_v2.csv', nrows = 1)
test

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911


,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0


,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,20150513,0,0,0,0,1,1,280.335


,msno,is_churn
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1


,msno,is_churn
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0


In [6]:
id_to_hash(members.loc[0, 'msno'])

209512247756457468966515739358104959027

In [7]:
import os

base_dir = '/data/churn/parts/'

# Create 1000 directories for each partition
for i in range(1000):
    os.makedirs(base_dir + f'p{i}', exist_ok=True)
    
len(os.listdir(base_dir))

1000

In [8]:
','.join(list(transactions.columns))

'msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel'

## Create Files

In each partition there are 5 csv files. The following code writes the header for each of the five files in each of the `N_PARTITION` partitions.

In [11]:
InteractiveShell.ast_node_interactivity = 'last_expr'

In [59]:
def create_blank_partitions():
    # For each partition create the files with headers
    for i in range(N_PARTITIONS):
        directory = base_dir + f'p{i}/'
        # Create five files
        for file in ['transactions.csv', 'train.csv', 'test.csv', 'members.csv', 'logs.csv']:
            # Write file header as first line
            with open(directory + file, 'w') as f:
                if file == 'transactions.csv':
                    f.write(','.join(list(transactions.columns)))
                elif file == 'train.csv':
                    f.write(','.join(list(train.columns)))
                elif file == 'test.csv':
                    f.write(','.join(list(train.columns)))
                elif file == 'members.csv':
                    f.write(','.join(list(members.columns)))
                elif file == 'logs.csv':
                    f.write(','.join(list(logs.columns)))
                    
    return directory

directory = create_blank_partitions()

In [60]:
os.listdir(directory)

['transactions.csv', 'members.csv', 'test.csv', 'train.csv', 'logs.csv']

### Example of Writing a Row

For each file, we'll iterate over the rows one at a time. The process for handling a row is:

1. Convert the customer id to an integer by hashing
2. Convert the integer to a partition number by modulus dividing by the number of partitions
3. Append the row to the correct partition file

Let's see how this works with a single row.

In [38]:
# Iterate through the dataframe one row at a time
for i, row in members.iterrows():
    # Find the partition by hashing the id
    partition = id_to_hash(row['msno']) % N_PARTITIONS
    # Open the file for appending
    with open(base_dir + f'p{partition}/members.csv', 'a') as f:
        # Write a newline and then the information
        f.write('\n')
        f.write(','.join([str(x) for x in row.values]))
    if i > 1:
        break

In [39]:
pd.read_csv(base_dir + f'p{partition}/members.csv')

,msno,city,bd,gender,registered_via,registration_init_time
0,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,20110915


Everything looks like it went well with the first attempt. Now we'll run this on a complete dataset.

## For Real

In [40]:
from timeit import default_timer as timer

### Member Information

We'll start off with the members. 

In [41]:
members = pd.read_csv('/data/churn/members_v3.csv')
members.shape

(6769473, 6)

## Iterrows

The first implementation will try using `iterrows`. This is fairly slow because Pandas packages the row as a Pandas series before iteration.

In [19]:
start = timer()

for i, row in members.iterrows():
    # Find the partition number by hashing the id
    partition = id_to_hash(row['msno']) % N_PARTITIONS
    
    # Open file for appending
    with open(base_dir + f'p{partition}/members.csv', 'a') as f:
        # Write a new line and then data
        f.write('\n')
        f.write(','.join([str(x) for x in row.values]))
        
    if i % 10000 == 0:
        print(f'{100 * round(i / members.shape[0], 2)}% complete. {round(timer() - start)} seconds elapsed.', end = '\r')

end = timer()
print(f'Reading and Writing {i} lines took {round(end - start)} seconds using iterrows.')

Reading and Writing 6769472 lines took 1456 seconds.


In [20]:
pd.read_csv(base_dir + f'p{partition}/members.csv').head()

,msno,city,bd,gender,registered_via,registration_init_time
0,+zMKqjvsTvD7O0Fvntk3VXe4ovwvD4KYk6PJZ92Ky60=,1,0,NaN,9,20161227
1,3p2AY1tZAYa4LFcs0/plkuPv2hY9smh/xgcbKjtU9Dc=,5,26,male,3,20141109
2,BO0XUBzHeItkHI3N5g4uL08Ld1T/ZW/8GrbjBmT4s3w=,1,0,NaN,7,20161228
3,qy7PNK2EE4+x6xeIdqjFVw5FlmxnFKylv6LKqqGbSo4=,1,23,female,4,20170113
4,c36721uHBQyhoVko21J9rR44Fex2ul72a74k0M7IkiQ=,1,0,NaN,4,20170121


## Itertuples

Itertuples should be faster than iterrows because Pandas packages the row as a tuple instead of as a series. 

In [61]:
_ = create_blank_partitions()

In [55]:
start = timer()

for i, row in enumerate(members.itertuples()):
    
    # Find the partition number by hashing the id
    partition = id_to_hash(row[1]) % N_PARTITIONS
    
    # Open file for appending
    with open(base_dir + f'p{partition}/members.csv', 'a') as f:
        # Write a new line and then data
        f.write('\n')
        f.write(','.join([str(x) for x in row[1:]]))
        
    if i % 10000 == 0:
        print(f'{100 * round(i / members.shape[0], 2)}% complete. {round(timer() - start)} seconds elapsed.', end = '\r')

end = timer()
print(f'Reading and Writing {i} lines took {round(end - start)} seconds using itertuples.')

Reading and Writing 6769472 lines took 366 seconds using iterrows.


In [62]:
pd.read_csv(base_dir + f'p{partition}/members.csv').head()

,msno,city,bd,gender,registered_via,registration_init_time


## Apply

The other operation that could be used is apply. To use `apply`, we write a small function that saves the row and then call apply to the dataframe using `axis = 1` which sends each row to the function.

In [63]:
def save_row(row, name):
    # Find the partition number by hashing the id
    partition = id_to_hash(row['msno']) % N_PARTITIONS
    
    # Open file for appending
    with open(base_dir + f'p{partition}/{name}.csv', 'a') as f:
        # Write a new line and then data
        f.write('\n')
        f.write(','.join([str(x) for x in row.values]))

In [ ]:
from tqdm import tqdm_notebook
from tqdm import tqdm
tqdm.pandas()

start = timer()
members.progress_apply(save_row, axis = 1, name = 'members')
end = timer()

print(f'Writing {members.shape[0]} rows took {round(end - start)} seconds using apply.')



  0%|          | 0/6769473 [00:00<?, ?it/s]

  0%|          | 1/6769473 [00:02<4079:47:15,  2.17s/it]

  0%|          | 518/6769473 [00:02<2855:44:32,  1.52s/it]

  0%|          | 1159/6769473 [00:02<1998:55:06,  1.06s/it]

  0%|          | 1739/6769473 [00:02<1399:13:12,  1.34it/s]

  0%|          | 2635/6769473 [00:02<979:23:15,  1.92it/s] 

  0%|          | 3480/6769473 [00:02<685:33:08,  2.74it/s]

  0%|          | 4285/6769473 [00:02<479:53:58,  3.92it/s]

  0%|          | 5264/6769473 [00:02<335:56:19,  5.59it/s]

  0%|          | 6453/6769473 [00:02<235:09:47,  7.99it/s]

  0%|          | 7350/6769473 [00:03<164:39:35, 11.41it/s]

  0%|          | 8229/6769473 [00:03<115:19:04, 16.29it/s]

  0%|          | 9083/6769473 [00:03<80:46:48, 23.25it/s] 

  0%|          | 10027/6769473 [00:03<56:35:52, 33.17it/s]

  0%|          | 10903/6769473 [00:03<39:41:25, 47.30it/s]

  0%|          | 11851/6769473 [00:03<27:50:19, 67.43it/s]

  0%|          | 12741/6769473 [00:03<19:32:52, 96.0

  2%|▏         | 112282/6769473 [00:15<12:30, 8873.76it/s]

  2%|▏         | 113187/6769473 [00:15<13:08, 8436.64it/s]

  2%|▏         | 114048/6769473 [00:15<13:35, 8159.39it/s]

  2%|▏         | 114878/6769473 [00:15<14:09, 7833.11it/s]

  2%|▏         | 115674/6769473 [00:15<14:39, 7565.41it/s]

  2%|▏         | 116442/6769473 [00:15<15:40, 7070.73it/s]

  2%|▏         | 117163/6769473 [00:16<16:44, 6625.51it/s]

  2%|▏         | 118030/6769473 [00:16<15:33, 7129.09it/s]

  2%|▏         | 119075/6769473 [00:16<14:04, 7878.71it/s]

  2%|▏         | 119916/6769473 [00:16<13:48, 8029.53it/s]

  2%|▏         | 120928/6769473 [00:16<12:56, 8558.54it/s]

  2%|▏         | 121814/6769473 [00:16<13:39, 8109.54it/s]

  2%|▏         | 122651/6769473 [00:16<14:08, 7830.44it/s]

  2%|▏         | 123558/6769473 [00:16<13:34, 8162.94it/s]

  2%|▏         | 124409/6769473 [00:16<13:24, 8263.57it/s]

  2%|▏         | 125305/6769473 [00:16<13:05, 8460.37it/s]

  2%|▏         | 126183/6769473 [00:17<1

  3%|▎         | 222374/6769473 [00:28<12:31, 8710.67it/s]

  3%|▎         | 223349/6769473 [00:28<12:07, 8996.05it/s]

  3%|▎         | 224260/6769473 [00:28<12:10, 8964.52it/s]

  3%|▎         | 225165/6769473 [00:28<12:30, 8723.03it/s]

  3%|▎         | 226045/6769473 [00:28<12:46, 8536.33it/s]

  3%|▎         | 226906/6769473 [00:29<12:44, 8557.67it/s]

  3%|▎         | 227766/6769473 [00:29<13:35, 8020.14it/s]

  3%|▎         | 228624/6769473 [00:29<13:19, 8178.43it/s]

  3%|▎         | 229450/6769473 [00:29<13:42, 7955.56it/s]

  3%|▎         | 230253/6769473 [00:29<14:00, 7776.40it/s]

  3%|▎         | 231037/6769473 [00:29<14:12, 7673.38it/s]

  3%|▎         | 231864/6769473 [00:29<13:53, 7842.12it/s]

  3%|▎         | 232653/6769473 [00:29<14:03, 7748.17it/s]

  3%|▎         | 233615/6769473 [00:29<13:14, 8227.38it/s]

  3%|▎         | 234599/6769473 [00:29<12:35, 8651.44it/s]

  3%|▎         | 235478/6769473 [00:30<12:39, 8607.02it/s]

  3%|▎         | 236348/6769473 [00:30<1

  5%|▍         | 332409/6769473 [00:41<13:55, 7701.81it/s]

  5%|▍         | 333196/6769473 [00:41<13:50, 7751.39it/s]

  5%|▍         | 333975/6769473 [00:41<13:57, 7688.59it/s]

  5%|▍         | 334747/6769473 [00:42<14:35, 7351.33it/s]

  5%|▍         | 335652/6769473 [00:42<13:46, 7788.79it/s]

  5%|▍         | 336442/6769473 [00:42<15:41, 6831.32it/s]

  5%|▍         | 337370/6769473 [00:42<14:27, 7417.06it/s]

  5%|▍         | 338216/6769473 [00:42<13:55, 7701.82it/s]

  5%|▌         | 339014/6769473 [00:42<14:24, 7438.95it/s]

  5%|▌         | 339839/6769473 [00:42<13:58, 7664.48it/s]

  5%|▌         | 340622/6769473 [00:42<14:36, 7332.30it/s]

  5%|▌         | 341370/6769473 [00:42<14:58, 7150.69it/s]

  5%|▌         | 342247/6769473 [00:43<14:09, 7568.58it/s]

  5%|▌         | 343019/6769473 [00:43<14:04, 7608.40it/s]

  5%|▌         | 343930/6769473 [00:43<13:22, 8002.57it/s]

  5%|▌         | 344799/6769473 [00:43<13:03, 8195.21it/s]

  5%|▌         | 345658/6769473 [00:43<1

  7%|▋         | 441797/6769473 [00:54<14:23, 7329.51it/s]

  7%|▋         | 442569/6769473 [00:54<14:10, 7440.66it/s]

  7%|▋         | 443371/6769473 [00:55<13:52, 7603.21it/s]

  7%|▋         | 444140/6769473 [00:55<14:01, 7517.55it/s]

  7%|▋         | 444941/6769473 [00:55<14:12, 7420.72it/s]

  7%|▋         | 445688/6769473 [00:55<16:35, 6354.27it/s]

  7%|▋         | 446398/6769473 [00:55<16:03, 6560.80it/s]

  7%|▋         | 447077/6769473 [00:55<16:27, 6400.76it/s]

  7%|▋         | 447971/6769473 [00:55<15:03, 6995.65it/s]

  7%|▋         | 448791/6769473 [00:55<14:23, 7316.89it/s]

  7%|▋         | 449547/6769473 [00:56<15:39, 6726.46it/s]

  7%|▋         | 450246/6769473 [00:56<16:02, 6565.73it/s]

  7%|▋         | 450932/6769473 [00:56<15:50, 6649.64it/s]

  7%|▋         | 451812/6769473 [00:56<14:40, 7174.44it/s]

  7%|▋         | 452551/6769473 [00:56<15:11, 6926.50it/s]

  7%|▋         | 453261/6769473 [00:56<15:38, 6733.47it/s]

  7%|▋         | 454170/6769473 [00:56<1

...

 45%|████▌     | 3063376/6769473 [06:09<08:20, 7398.33it/s]

 45%|████▌     | 3064234/6769473 [06:09<08:00, 7715.22it/s]

 45%|████▌     | 3065016/6769473 [06:09<08:11, 7542.20it/s]

 45%|████▌     | 3065778/6769473 [06:09<08:29, 7264.75it/s]

 45%|████▌     | 3066512/6769473 [06:09<08:32, 7229.69it/s]

 45%|████▌     | 3067308/6769473 [06:09<08:18, 7433.23it/s]

 45%|████▌     | 3068242/6769473 [06:09<07:47, 7917.81it/s]

 45%|████▌     | 3069081/6769473 [06:09<07:39, 8051.99it/s]

 45%|████▌     | 3070128/6769473 [06:10<07:07, 8651.07it/s]

 45%|████▌     | 3071019/6769473 [06:10<07:03, 8726.34it/s]

 45%|████▌     | 3071906/6769473 [06:10<07:23, 8336.47it/s]

 45%|████▌     | 3072892/6769473 [06:10<07:02, 8740.93it/s]

 45%|████▌     | 3073807/6769473 [06:10<06:57, 8859.50it/s]

 45%|████▌     | 3074706/6769473 [06:10<06:55, 8897.54it/s]

 45%|████▌     | 3075604/6769473 [06:10<06:54, 8911.65it/s]

 45%|████▌     | 3076501/6769473 [06:10<07:17, 8442.39it/s]

 45%|████▌     | 3077584

 47%|████▋     | 3175528/6769473 [06:22<07:26, 8042.24it/s]

 47%|████▋     | 3176423/6769473 [06:22<07:13, 8294.37it/s]

 47%|████▋     | 3177407/6769473 [06:22<06:52, 8704.23it/s]

 47%|████▋     | 3178295/6769473 [06:22<06:56, 8619.37it/s]

 47%|████▋     | 3179170/6769473 [06:22<07:04, 8461.61it/s]

 47%|████▋     | 3180026/6769473 [06:22<07:08, 8379.05it/s]

 47%|████▋     | 3180871/6769473 [06:23<07:24, 8072.56it/s]

 47%|████▋     | 3181991/6769473 [06:23<06:47, 8809.11it/s]

 47%|████▋     | 3182964/6769473 [06:23<06:35, 9065.13it/s]

 47%|████▋     | 3183908/6769473 [06:23<06:30, 9172.07it/s]

 47%|████▋     | 3184882/6769473 [06:23<06:24, 9333.80it/s]

 47%|████▋     | 3185829/6769473 [06:23<06:22, 9372.94it/s]

 47%|████▋     | 3186774/6769473 [06:23<06:27, 9236.16it/s]

 47%|████▋     | 3187704/6769473 [06:23<06:46, 8809.60it/s]

 47%|████▋     | 3188593/6769473 [06:23<06:58, 8562.54it/s]

 47%|████▋     | 3189457/6769473 [06:23<07:01, 8488.01it/s]

 47%|████▋     | 3190311

 49%|████▊     | 3287957/6769473 [06:35<07:14, 8007.28it/s]

 49%|████▊     | 3288930/6769473 [06:35<06:51, 8454.05it/s]

 49%|████▊     | 3289787/6769473 [06:35<06:51, 8457.47it/s]

 49%|████▊     | 3290743/6769473 [06:35<06:37, 8758.32it/s]

 49%|████▊     | 3291627/6769473 [06:35<06:52, 8440.99it/s]

 49%|████▊     | 3292545/6769473 [06:35<06:45, 8583.87it/s]

 49%|████▊     | 3293410/6769473 [06:36<07:19, 7916.84it/s]

 49%|████▊     | 3294217/6769473 [06:36<07:20, 7890.95it/s]

 49%|████▊     | 3295180/6769473 [06:36<06:56, 8342.08it/s]

 49%|████▊     | 3296029/6769473 [06:36<06:59, 8284.74it/s]

 49%|████▊     | 3296957/6769473 [06:36<06:45, 8558.68it/s]

 49%|████▊     | 3297923/6769473 [06:36<06:31, 8860.03it/s]

 49%|████▊     | 3299004/6769473 [06:36<06:10, 9366.74it/s]

 49%|████▉     | 3300213/6769473 [06:36<05:45, 10043.24it/s]

 49%|████▉     | 3301241/6769473 [06:36<05:51, 9868.66it/s] 

 49%|████▉     | 3302246/6769473 [06:36<05:50, 9904.17it/s]

 49%|████▉     | 33032

 50%|█████     | 3397935/6769473 [06:48<07:10, 7823.09it/s]

 50%|█████     | 3398734/6769473 [06:48<07:19, 7674.16it/s]

 50%|█████     | 3399565/6769473 [06:48<07:09, 7853.84it/s]

 50%|█████     | 3400619/6769473 [06:48<06:36, 8503.43it/s]

 50%|█████     | 3401492/6769473 [06:48<06:35, 8515.46it/s]

 50%|█████     | 3402462/6769473 [06:49<06:20, 8838.71it/s]

 50%|█████     | 3403360/6769473 [06:49<06:37, 8464.19it/s]

 50%|█████     | 3404285/6769473 [06:49<06:27, 8682.76it/s]

 50%|█████     | 3405164/6769473 [06:49<06:30, 8617.69it/s]

 50%|█████     | 3406281/6769473 [06:49<06:03, 9251.15it/s]

 50%|█████     | 3407261/6769473 [06:49<05:57, 9408.07it/s]

 50%|█████     | 3408216/6769473 [06:49<06:01, 9298.71it/s]

 50%|█████     | 3409156/6769473 [06:49<06:06, 9161.01it/s]

 50%|█████     | 3410080/6769473 [06:49<06:06, 9160.05it/s]

 50%|█████     | 3411002/6769473 [06:49<06:20, 8826.38it/s]

 50%|█████     | 3411947/6769473 [06:50<06:12, 9004.27it/s]

 50%|█████     | 3412962

 52%|█████▏    | 3498043/6769473 [07:01<10:16, 5304.67it/s]

 52%|█████▏    | 3498624/6769473 [07:02<10:15, 5314.21it/s]

 52%|█████▏    | 3499458/6769473 [07:02<09:08, 5961.13it/s]

 52%|█████▏    | 3500359/6769473 [07:02<08:12, 6633.67it/s]

 52%|█████▏    | 3501082/6769473 [07:02<08:41, 6271.01it/s]

 52%|█████▏    | 3501892/6769473 [07:02<08:05, 6724.91it/s]

 52%|█████▏    | 3502658/6769473 [07:02<07:48, 6979.37it/s]

 52%|█████▏    | 3503583/6769473 [07:02<07:13, 7533.72it/s]

 52%|█████▏    | 3504754/6769473 [07:02<06:27, 8435.27it/s]

 52%|█████▏    | 3505739/6769473 [07:02<06:10, 8813.87it/s]

 52%|█████▏    | 3506667/6769473 [07:03<06:24, 8484.08it/s]

 52%|█████▏    | 3507645/6769473 [07:03<06:09, 8833.97it/s]

 52%|█████▏    | 3508770/6769473 [07:03<05:45, 9441.98it/s]

 52%|█████▏    | 3509746/6769473 [07:03<06:00, 9033.96it/s]

 52%|█████▏    | 3510676/6769473 [07:03<06:17, 8625.46it/s]

 52%|█████▏    | 3511561/6769473 [07:03<06:21, 8533.56it/s]

 52%|█████▏    | 3512630

In [ ]:
pd.read_csv(base_dir + f'p{partition}/members.csv').head()

## Groupby 

Another option is to compute the partitions all at once, groupby the partition, and write the grouped dataframe to disk.

## Reusable Hashing Dataframe Function

To make the process reusable, we'll write a function that does this for us. It will take in a dataframe, a name for the file to save the lines to, and an optional progress argument. The partition is determined by hashing the customer id and then modulo dividing by the number of partitions. Each line of the dataframe will be saved to the appropriate file in the partition.

In [22]:
def partition_by_hashing(df, name, progress = None):
    """Partition a dataframe into N_PARTITIONS by hashing the id.
    
    Params
    --------
        df (pandas dataframe): dataframe for partition. Must have 'msno' column.
        name (str): name of dataframe. Used for saving the row data.
        progress (int, optional): number of rows to be processed before displaying information.
                                  Defaults to None
                                  
    Returns:
    --------
        Nothing returned. Dataframe is saved one line at a time as csv files to the N_PARTITIONS 
    """
    
    start = timer()
    
    # Iterate through one row at a time
    for i, row in df.iterrows():
        # Find the partition number by hashing the id
        partition = id_to_hash(row['msno']) % N_PARTITIONS
        
        # Open file for appending
        with open(base_dir + f'p{partition}/{name}.csv', 'a') as f:
            # Write a new line and then data
            f.write('\n')
            f.write(','.join([str(x) for x in row.values]))
            
        # Record progress every `progress` steps
        if progress is not None:
            if i % progress == 0:
                print(f'{100 * round(i / df.shape[0], 2)}% complete. {round(timer() - start)} seconds elapsed.')
    
    end = timer()
    
    print(f'{i} rows processed in {round(end - start)} seconds.')

## Training Data

Now we can use this function to partition the training data.